In [1]:
import pickle
import numpy as np
from matplotlib import pyplot as plt
import random
import sys
import seaborn as sns
import pandas as pd
from scipy import stats
import os
from sklearn.mixture import GaussianMixture
import math

SyntaxError: invalid syntax (core.py, line 153)

In [3]:
def get_data_in_dataframe(function_to_ages,ages_list):
    data = [] #function, ages_list, mean, std, len, rand_mean, rand_std
    for func in function_to_ages:
        curr = []
        ages = np.array(function_to_ages[func])
        
        #remove Nones
        ages = ages[ages != np.array(None)]
        if len(ages) == 0:
            continue
            
        #calculation of stats
        curr.append(func)
        curr.append(ages)
        curr.append(np.mean(ages))
        curr.append(np.std(ages))
        curr.append(len(ages))
        
        g = GaussianMixure(2)
        covs = g.fit(ages.reshape(-1,1)) 
        curr.append(math.sqrt(covs[0][0][0]))
        curr.append(math.sqrt(covs[1][0][0]))
            
        #random (based on num ages)
        rand_ages = random.sample(ages_list,len(ages))
        curr.append(np.mean(rand_ages))
        curr.append(np.std(rand_ages))
        g = GaussianMixure(2)
        covs = g.fit(rand_ages.reshape(-1,1)) 
        curr.append(math.sqrt(covs[0][0][0]))
        curr.append(math.sqrt(covs[1][0][0]))
        data.append(curr)
        
    df = pd.DataFrame(data)
    df.columns = ['function_name','ages_list','age_mean','age_std','num_els','age_gmm2_std_1','age_gmm2_std_2','rand_age_mean','rand_age_std','rand_age_gmm2_std_1','rand_age_gmm2_std_2']
    return df

In [5]:
# GET FUNC GROUPED AGE DATA
function_to_ages = pickle.load(open("data/function_to_age_simplified_dict.p","rb"))

gene_to_age_dict = pickle.load(open("data/gene_to_age_simplified_dict.p","rb"))
ages_list = []
for gene in gene_to_age_dict:
    ages_list.append(gene_to_age_dict[gene])
    
df = get_data_in_dataframe(function_to_ages,ages_list)

NameError: name 'GaussianMixure' is not defined

In [ ]:
df